# Runs Cytocipher using scanpy scores on the full Tabula Sapiens data.
                     
# INPUT: 

    * data/test_data/TabulaSapiens_Cytociphered.h5ad

# OUTPUT: 

    * data/test_data/scanpy_tabula-full_results.pkl

# Environment Setup

In [1]:
work_dir = './' # Change to where cloned Cytocipher_manuscript/
import os
os.chdir(work_dir)

import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib
import matplotlib.pyplot as plt

import utils.visualisation.helpers as vhs
import utils.visualisation.quick_plots as qpl
qpl.setUp()

import seaborn as sb
from matplotlib_venn import venn2

pre_dir = '/media/WorkingSpace/Share/hypo_atlas/'
data_dir = pre_dir+'data/test_data/'
out_dir = data_dir

import cytocipher as cc
import importlib as imp

import cytocipher.score_and_merge.cluster_merge as cm
import cytocipher.score_and_merge.cluster_score as cs
imp.reload(cm)
imp.reload(cs)

import utils.preprocessing.load_data.simple_pickle as spl

# Loading the data

In [2]:
method = 'scanpy'

In [3]:
data = sc.read_h5ad(data_dir+'TabulaSapiens_Cytociphered.h5ad')

In [4]:
data.shape

(483152, 58870)

## Now let's run using Code-scoring to get p-values..

In [5]:
import psutil

def get_process_memory():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss

In [24]:
def scanpy_merge(data, cluster_key, marker_key,
                 k=15, mnn_frac_cutoff=None, random_state=20, p_cut=0.01, score_group_method='quantiles',
                 p_adjust=True, p_adjust_method='fdr_bh'):
    """ Extra marker calling in order to be in-line with the cc.tl.merge_clusters function.
    """
    cc.tl.get_markers(data, cluster_key, var_groups='highly_variable')
    
    ### Scanpy-scoring.
    scanpy_scores = np.zeros((data.shape[0], len(data.uns[marker_key])))
    for i, group in enumerate(data.uns[marker_key]):
        sc.tl.score_genes(data, data.uns[marker_key][group])
        scanpy_scores[:,i] = data.obs['score'].values
    scanpy_scores = pd.DataFrame(scanpy_scores, index=data.obs_names, columns = list(data.uns[marker_key].keys()))
    data.obsm[f'{cluster_key}_enrich_scores'] = scanpy_scores
    
    ### Merging by Scanpy-score.
    cm.merge_clusters_single(data, cluster_key, f'{cluster_key}_merged',
                            k=k, mnn_frac_cutoff=mnn_frac_cutoff, random_state=random_state,
                          p_cut=p_cut,
                          score_group_method=score_group_method,
                          p_adjust=p_adjust, p_adjust_method=p_adjust_method,
                          verbose=False)
    
    cc.tl.get_markers(data, f'{cluster_key}_merged', var_groups='highly_variable')
    
    ### Scanpy-scoring Final round for merge.
    scanpy_scores = np.zeros((data.shape[0], len(data.uns[f'{cluster_key}_merged_markers'])))
    for i, group in enumerate(data.uns[f'{cluster_key}_merged_markers']):
        sc.tl.score_genes(data, data.uns[f'{cluster_key}_merged_markers'][group])
        scanpy_scores[:,i] = data.obs['score'].values
    scanpy_scores = pd.DataFrame(scanpy_scores, index=data.obs_names, 
                                 columns = list(data.uns[f'{cluster_key}_merged_markers'].keys()))
    data.obsm[f'{cluster_key}_merged_enrich_scores'] = scanpy_scores

In [ ]:
##### Now re-running with the coexpr scoring...
import time
import tracemalloc
start = time.time()
start_mem = get_process_memory()
tracemalloc.start()
# cc.tl.merge_clusters(data, 'overclusters', var_groups='highly_variable', n_cpus=15, max_iter=0,
#                      enrich_method='code', 
#                       k=None, squash_exception=True,
#                      )
scanpy_merge(data, 'overclusters', 'overclusters_markers', k=None)
file_ = open(data_dir+f'tabula_{method}-full-mem.txt', 'w')
print(tracemalloc.get_traced_memory(), file=file_)
tracemalloc.stop()
end = time.time()
end_mem = get_process_memory()

In [ ]:
lapse = end-start
print("Time in seconds: ", lapse, file=file_)
lapse_mem = end_mem - start_mem
print("Memory used: ", lapse_mem, file=file_)
file_.close()

In [38]:
!cat {data_dir}tabula_{method}-full-mem.txt

(4345451145, 15774847579)
Time in seconds:  13662.974824905396
Memory used:  4156055552


In [41]:
(15774847579-4345451145)/1e9 #### Memory in Gb

11.429396434

In [40]:
13662.974824905396/60/60 #### Time in hours

3.7952707846959433

In [ ]:
##### Cacheing results from Coexpr-scoring
enrich = method
results = {'enrich': enrich}
results[f'overclusters_{enrich}_scores'] = data.obsm[f'overclusters_enrich_scores']
results[f'overclusters_merged_{enrich}_scores'] = data.obsm[f'overclusters_merged_enrich_scores']
results[f'overclusters_{enrich}_merged'] = data.obs['overclusters_merged']
uns_keys = [key for key in data.uns.keys() if key.startswith('overclusters_')]
for key in uns_keys:
    results[f'{key}_{enrich}'] = data.uns[key]

In [ ]:
cc.pl.enrich_heatmap(data, 'overclusters', per_cell=False, scale_cols=False, scale_rows=True)

In [ ]:
### Saving these results as a pickle object
spl.saveAsPickle(data_dir+f'{enrich}_tabula-full_results.pkl', results)